<a href="https://colab.research.google.com/github/javajaewook/2020-2_NLP/blob/master/Check_4_Conditions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install konlpy

# 임포트

In [7]:
# 혹시 모르니 다 넣어놓는 import
import pandas as pd
import csv
import numpy as np
from konlpy.tag import Okt, Kkma
import urllib.request
import matplotlib.pyplot as plt
import re
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
import copy
import nltk

%matplotlib inline

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 분절

In [9]:
# 초성 리스트. 00 ~ 18
CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
# 중성 리스트. 00 ~ 20
JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
# 종성 리스트. 00 ~ 27 + 1(1개 없음)
JONGSUNG_LIST = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

def korean_split(korean_word):
    r_lst = []
    for w in list(korean_word.strip()):
        ## 영어인 경우 구분해서 작성함. 
        if '가'<=w<='힣':
            ## 588개 마다 초성이 바뀜. 
            ch1 = (ord(w) - ord('가'))//588
            ## 중성은 총 28가지 종류
            ch2 = ((ord(w) - ord('가')) - (588*ch1)) // 28
            ch3 = (ord(w) - ord('가')) - (588*ch1) - 28*ch2
            r_lst.append([CHOSUNG_LIST[ch1], JUNGSUNG_LIST[ch2], JONGSUNG_LIST[ch3]])
        else:
            r_lst.append([w])
    return r_lst


# 어절 분석

분절은 korean_split(word)로 하면 됨

## ㅇ 첨가
조건 : 음절의 종성이 &nbsp;'ㅇ'<br>
어절의 마지막 음절이 조건을 만족하는 어절을 추출 후 확인

In [10]:
def o_addition(word):
    '''
    Variables
        word (String Literal, Argument) : 입력 한글 단어
        flag (Boolean, Local) : ㅇ 첨가 규칙 만족 여부
        syllable (List[List[char]], Local) : 단어의 분절 결과

    Return
        flag (Boolean, Local) : word의 ㅇ 첨가 규칙 만족 여부
    '''
    flag = False 
    syllable = korean_split(word)

    target = syllable[-1][2] # 마지막 음절의 종성

    if target == 'ㅇ':
        flag = True
    
    return flag

In [11]:
print('ㅇ 첨가\ncase 1: {} -> {}\ncase 2: {} -> {}'.format('용용죽겠지', o_addition('용용죽겠지'),
                                                  '용용죽겠징', o_addition('용용죽겠징')))

ㅇ 첨가
case 1: 용용죽겠지 -> False
case 2: 용용죽겠징 -> True


## 축약
조건 : 음절의 중성, &nbsp;종성이 존재함 <br>
어절의 모든 음절이 조건을 만족하는 어절을 추출 후 확인<br><br>

-문제점-<br>
조건을 만족하지 않는 축약어가 다수 존재함.<br><br>

-규칙을 만족하지 않는 축약어 예시-<br>
존버 :&nbsp; '존나 버티다'&nbsp; 의 축약어<br>
오저치고 :&nbsp; '오늘 저녁 치킨 고'&nbsp; 의 축약어




In [12]:
def abbreviation(word):
    '''
    Variables
        word (String Literal, Argument) : 입력 한글 단어
        flag (Boolean, Local) : 축약 규칙 만족 여부
        syllable (List[List[char]], Local) : 단어의 분절 결과

    Return
        flag (Boolean, Local) : word의 축약 규칙 만족 여부
    '''
    flag = True
    syllable = korean_split(word)

    for syl in syllable:
        target_1 = syl[1]
        target_2 = syl[2]
        if (target_1 == ' ' or target_2 == ' '):
            flag = False
            break;
        else : continue
    
    return flag

In [13]:
print('축약\ncase 1: {} -> {}\ncase 2: {} -> {}'.format('일욜날', abbreviation('일욜날'), 
                                                  '일요일날', abbreviation('일요일날')))

축약
case 1: 일욜날 -> True
case 2: 일요일날 -> False


In [14]:
print('축약(반례)\ncase 1: {} -> {}\ncase 2: {} -> {}'.format('파리바게트', abbreviation('파리바게트'), 
                                                  '파바', abbreviation('파바')))

축약(반례)
case 1: 파리바게트 -> False
case 2: 파바 -> False


## ㅛ -> ㅕ
조건 : 어절의 중성이&nbsp; 'ㅕ' +&nbsp; 어절의 종성이 &nbsp;'&emsp;'<br>
어절의 마지막 음절이 조건을 만족하는 어절을 추출 후 확인

In [15]:
def yo_to_yeo(word):
    '''
    Variables
        word (String Literal, Argument) : 입력 한글 단어
        flag (Boolean, Local) : ㅛ->ㅕ 규칙 만족 여부
        syllable (List[List[char]], Local) : 단어의 분절 결과

    Return
        flag (Boolean, Local) : word의 ㅛ->ㅕ 규칙 만족 여부
    '''
    flag = False
    syllable = korean_split(word)

    target_1 = syllable[-1][1]
    target_2 = syllable[-1][2]

    if (target_1 == 'ㅕ' and target_2 == ' '):
        flag = True
    
    return flag

In [16]:
print('ㅛ -> ㅕ\ncase 1: {} -> {}\ncase 2: {} -> {}'.format('좋아여', yo_to_yeo('좋아여'),
                                                  '좋아요', yo_to_yeo('좋아요')))

ㅛ -> ㅕ
case 1: 좋아여 -> True
case 2: 좋아요 -> False


## ㅜ 
조건 : 어절의 중성이 &nbsp;'ㅜ' &nbsp;+ &nbsp;어절의 종성이 &nbsp;'&emsp;'<br>

case 1) 어절의 마지막 음절이 조건을 만족

case 2) 어절 첫 음절이 조건을 만족

In [17]:
def woo(word):
    '''
    Variables
        word (String Literal, Argument) : 입력 한글 단어
        flag (Boolean, Local) : ㅜ 규칙 만족 여부
        syllable (List[List[char]], Local) : 단어의 분절 결과

    Return
        flag (Boolean, Local) : word의 ㅜ 규칙 만족 여부
    '''
    flag = False
    syllable = korean_split(word)

    # case 1
    target_1 = syllable[-1][1]
    target_2 = syllable[-1][2]

    if (target_1 == 'ㅜ' and target_2 == ' '):
        flag = True

    # case 2
    target_1 = syllable[0][1]
    target_2 = syllable[0][2]

    if (target_1 == 'ㅜ' and target_2 == ' '):
        flag = True

    return flag

In [18]:
print('ㅜ\ncase 1: {} -> {}\ncase 2: {} -> {}'.format('좋구', woo('좋구'),
                                                  '좋고', woo('좋고')))

ㅜ
case 1: 좋구 -> True
case 2: 좋고 -> False


In [19]:
print('ㅜ\ncase 1: {} -> {}\ncase 2: {} -> {}'.format('구래', woo('구래'),
                                                  '그래', woo('그래')))

ㅜ
case 1: 구래 -> True
case 2: 그래 -> False


## 4가지 조건 분석
<br>

ㅇ 첨가, 축약, ㅛ->ㅕ, ㅜ 조건을 검사해서 리스트로 반환한다.

In [20]:
def check_4_conditions(word):
    '''
    Variables
        word (String Literal, Argument) : 입력 한글 단어
        flag_table (List[Boolean], Local) : word의 규칙 4종에 대한 각각의 만족 여부

    Return
        flag_table (List[Boolean], Local) : word의 규칙 [ㅇ첨가, 축약, ㅛ->ㅕ, ㅜ]에 대한 각각의 만족 여부.
    '''
    flag_table = []

    # ㅇ 첨가
    flag_table.append(o_addition(word))
    # 축약
    flag_table.append(abbreviation(word))
    # ㅛ -> ㅕ
    flag_table.append(yo_to_yeo(word))
    # ㅜ
    flag_table.append(woo(word))

    return flag_table

In [21]:
check_4_conditions('구랭')

[True, False, False, True]